In [ ]:

RAG system 5 steps:

1). data loading
2). split the data into chunks
3). create embedding for these chunks
4). store embedding in vector database
5). Retrieval the query related chunks from vector database


In [1]:
#install required pakages


!pip install langchain_community
!pip install openai
!pip install chromadb
!pip install tiktoken
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
#llm api keys

import os
from google.colab import userdata
#userdata.get('secretName')

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["COHERE_API_KEY"] = userdata.get('COHERE_API_KEY')


In [5]:
#step 1
#now load the your private data or document loading from any sources

!pip install pypdf --quiet



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.0 MB/s eta 0:00:00


In [7]:
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Agreements/EMPLOYEE_AGREEMENT.pdf

--2024-12-08 14:33:11--  https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Agreements/EMPLOYEE_AGREEMENT.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206079 (201K) [application/octet-stream]
Saving to: ‘EMPLOYEE_AGREEMENT.pdf’

EMPLOYEE_AGREEMENT. 100%[===================>] 201.25K  --.-KB/s    in 0.03s   

2024-12-08 14:33:12 (6.69 MB/s) - ‘EMPLOYEE_AGREEMENT.pdf’ saved [206079/206079]



In [8]:

from langchain.document_loaders import PyPDFLoader
loader=PyPDFLoader("EMPLOYEE_AGREEMENT.pdf")
pages=loader.load()
print(len(pages))

13


In [16]:



#steps2
#split data into chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter
#choose chuks size as 10% generally because of llm have limited no. of tokens at a time to generate the output
text_splitter=RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=30)
chunks=text_splitter.split_documents(pages)
print(len(chunks))


133


In [19]:


#steps 3
#convert chunks to embedding


from langchain.embeddings import OpenAIEmbeddings, CohereEmbeddings

embeddings=CohereEmbeddings(user_agent="langchain")
#embeddings=OpenAIEmbeddings()




In [13]:
!pip install chromadb

In [20]:


#steps 4
#store the embedding in vector database -->chromadb, faiss, eleastic-search

from langchain.vectorstores import Chroma
emp_rules_db= Chroma.from_documents(chunks,
                                    embeddings,
                                    persist_directory="emp_rules_db"
                          )
emp_rules_db.persist()


<ipython-input-20-32fae5dc7d67>:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  emp_rules_db.persist()


In [23]:


#step 5
#retrieval the chunks from vector database based on query embedding match


retriever = emp_rules_db.as_retriever()
result=retriever.get_relevant_documents("What is the policy for sick leaves",
                                        top_k=3)
result

[Document(metadata={'page': 1, 'source': 'EMPLOYEE_AGREEMENT.pdf'}, page_content='sick days and personal absence days as authorized by the Company for its other employees.\nARTICLE 6\nFringe Benefits\nEmployee shall be entitled to participate in and receive benefits from all of the Company’s employee\nbenefit plans that are now, or in the future may be, maintained by the'),
 Document(metadata={'page': 1, 'source': 'EMPLOYEE_AGREEMENT.pdf'}, page_content='5.2. Holidays.   Employee shall be entitled to the same paid holidays as authorized by the Company\nfor its other employees.\n5.3. Sick Days and Personal Absence Days.  Employee shall be entitled to the same number of paid'),
 Document(metadata={'page': 1, 'source': 'EMPLOYEE_AGREEMENT.pdf'}, page_content='ARTICLE 5\nVacations and Other Paid Absences\n5.1. Vacation Days.  Employee shall be entitled to the same paid vacation days each calendar year\nduring the term of this Employee Agreement as authorized by the Company for its other em

In [29]:
#4 document has retrieved based in query similarity

from langchain.retrievers import MultiQueryRetriever
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI, Cohere

llm=Cohere(temperature=0)

llm_based_retriever=MultiQueryRetriever.from_llm(llm=llm,
                                                 retriever = emp_rules_db.as_retriever(),
                                                )

<ipython-input-29-5128481ea426>:7: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  llm=Cohere(temperature=0)


In [40]:

#your own custom GPT works only on your private data
llm_based_retriever  #--->this is your custom GPT


MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['Chroma', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7ce45bb04850>, search_kwargs={}), llm_chain=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is \n    to generate 3 different versions of the given user \n    question to retrieve relevant documents from a vector  database. \n    By generating multiple perspectives on the user question, \n    your goal is to help the user overcome some of the limitations \n    of distance-based similarity search. Provide these alternative \n    questions separated by newlines. Original question: {question}')
| Cohere(client=<cohere.client.Client object at 0x7ce4594ee170>, async_client=<cohere.client.AsyncClient object at 0x7ce4583c1150>, temperature=0.0, cohere_api_key=SecretStr('**********'))
| LineListOutputParser())

In [41]:




from langchain.chains import RetrievalQA
#llm=OpenAI(temperature=0)
llm=Cohere(temperature=0)

Q_AChain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",# It takes a list of documents, inserts them all into a prompt
    retriever=llm_based_retriever
)

In [43]:

query="who is Akash?"
docs=Q_AChain({"query":query})
docs["result"]

INFO:langchain.retrievers.multi_query:Generated queries: ['Here are three alternative ways to phrase the question "who is Akash?":', '1. Provide information about the identity of Akash. ', '2. Who is this person named Akash? ', '3. Can you give details about the name Akash? ', "These variations hope to provide you with a more comprehensive collection of documents associated with the name Akash, as it slightly modifies the user's request."]


' I cannot find any reference to Akash in the provided context. \n\nCan I provide you with more information regarding the provided context? '